# TP RDD

Dans cet atelier, nous allons explorer certains les concepts de RDD que nous avons abordés. Nous utiliserons un ensemble de données comprenant les crimes signalés à Washington, du 3 octobre 2015 au 2 octobre 2016. Ces données proviennent du catalogue de données ouvertes du district de Columbia. Nous utiliserons ces données pour explorer certaines transformations et actions sur les RDD et les pairs RDD.
https://github.com/frankieliu/databricks2/blob/master/wash_dc_crime_incidents_2013.csv

## Pré-requis




Démarrer HDFS :

### Démarrer Spark

Pour démarrer votre cluster Spark, si cela n'a pas déjà été fait, vous devrez ouvrir un terminal puis exécuter la commande suivante :

Vérifiez que les containers smaster, sworker1 et sworker2 sont démarrés :

### Connexion à Spark

Etablissez une connexion au cluster Spark :


In [1]:
# Attention le driver ne peut instancier qu'un seul SparkContext
# Si vous exécuter plusieurs fois cette cellule vous obtiendrez une erreur:
# Cannot run multiple SparkContexts at once;
# Pensez bien à fermer le Sparkcontext à la fin de chaque TP avec la commande sc.stop()

from pyspark import SparkContext

sc = SparkContext("spark://smaster:7077", "TPSPARK")


Pour vérifier le point d'accés au cluster : 


In [2]:
sc.master

'spark://smaster:7077'

## Créer un RDD

La première étape consiste à charger les données. 
Exécutez la cellule suivante pour créer un RDD contenant les données.

In [3]:
myRdd =  sc.textFile("/dataspark/wash_dc_crime_incidents_2013.csv")

La lecture d'un fichier n'est pas une action et par conséquent aucun traitement sera effectué.

Lancer votre première action de comptage :

In [4]:
myRdd.count()

35898

## Exploration des données

Voyons quelques-unes des données.

In [5]:
# Affichez les 5 premières lignes du RDD "myRdd"
myRdd.take(5)

['CCN,REPORTDATETIME,SHIFT,OFFENSE,METHOD,LASTMODIFIEDDATE,BLOCKSITEADDRESS,BLOCKXCOORD,BLOCKYCOORD,WARD,ANC,DISTRICT,PSA,NEIGHBORHOODCLUSTER,BUSINESSIMPROVEMENTDISTRICT,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,START_DATE,END_DATE',
 '04104147,4/16/2013 12:00:00 AM,MIDNIGHT,HOMICIDE,KNIFE,8/7/2015 8:34:01 AM,1500 - 1599 BLOCK OF 1ST STREET SW,398943,133729,6,6D,FIRST,105,9,,006400 2,006400,Precinct 127,7/27/2004 8:30:00 PM,7/27/2004 8:30:00 PM',
 '05047867,6/5/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,KNIFE,8/7/2015 8:32:22 AM,6500  - 6599 BLOCK OF PINEY BRANCH ROAD NW,397769,144596,4,4B,FOURTH,402,17,,001901 4,001901,Precinct 59,4/15/2005 12:30:00 PM,',
 '07083463,7/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:32:15 AM,1800 - 1810 BLOCK OF COLUMBIA ROAD NW,396275,139402,1,1C,THIRD,303,1,ADAMS MORGAN,004002 1,004002,Precinct 25,7/14/2007 3:00:00 PM,',
 '09172197,4/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:33:35 AM,2322 - 2499 BLOCK OF ONTARIO ROAD NW,396518,139335,1,

Super, comme vous pouvez le constater il y a un en-tête. 
Cet en-tête est fourni au début du fichier, cela signifie qu'une seul partition disposera de l'en-tête.
Pour effectuer un traitement homogène sur l'ensemble des données, nous devrons vérifier si le bloc en cours de traitement a un en-tête.
Quand le fichier taille des Go voir de To de données, cette vérification peut vite devenir coûteuse, c'est pourquoi on préfère traiter les données sans en-tête.

Trouvez une solution pour supprimer l'en-tête :

Indice: utilisez la method filter de l'API RDD.

https://spark.apache.org/docs/latest/api/python/pyspark.html?

In [6]:
# indiquer le traitement à effectuer pour supprimer l'en-tête
# nous appelerons le RDD sans en-tête "noHeaderRdd" 

header = myRdd.first()
noHeaderRdd = myRdd.filter(lambda row : row != header) 
noHeaderRdd.first()

'04104147,4/16/2013 12:00:00 AM,MIDNIGHT,HOMICIDE,KNIFE,8/7/2015 8:34:01 AM,1500 - 1599 BLOCK OF 1ST STREET SW,398943,133729,6,6D,FIRST,105,9,,006400 2,006400,Precinct 127,7/27/2004 8:30:00 PM,7/27/2004 8:30:00 PM'

Sauvegarder les données sans l'entête sous le nom de fichier /data/temporary/wash_dc_crime_incidents_2013_without_header.csv

In [8]:
noHeaderRdd.saveAsTextFile('/dataspark/wash_dc_crime_incidents_2013_without_header.csv')

A partir du fichier sans entête, découper les lignes en cellules individuelles.

Nous appelerons le RDD resultant de cette transformation "CrimeData".


In [9]:
# Complétez le code
CrimeData= noHeaderRdd.map(lambda x : x.split(','))
CrimeData.take(2)

[['04104147',
  '4/16/2013 12:00:00 AM',
  'MIDNIGHT',
  'HOMICIDE',
  'KNIFE',
  '8/7/2015 8:34:01 AM',
  '1500 - 1599 BLOCK OF 1ST STREET SW',
  '398943',
  '133729',
  '6',
  '6D',
  'FIRST',
  '105',
  '9',
  '',
  '006400 2',
  '006400',
  'Precinct 127',
  '7/27/2004 8:30:00 PM',
  '7/27/2004 8:30:00 PM'],
 ['05047867',
  '6/5/2013 12:00:00 AM',
  'MIDNIGHT',
  'SEX ABUSE',
  'KNIFE',
  '8/7/2015 8:32:22 AM',
  '6500  - 6599 BLOCK OF PINEY BRANCH ROAD NW',
  '397769',
  '144596',
  '4',
  '4B',
  'FOURTH',
  '402',
  '17',
  '',
  '001901 4',
  '001901',
  'Precinct 59',
  '4/15/2005 12:30:00 PM',
  '']]

La réprésentation des données laisse à désirer sans la schéma des données.
En python, on peut associer des noms à chaque valeur du tuple avec le module namedtuple :

In [10]:
from collections import namedtuple

CrimeDataTuple = namedtuple('CrimeData', ['date_string', 'time_string', 'offense', 'latitude', 'longitude'])

def map_line(line):
  cols = line.split(",")
  return CrimeDataTuple(date_string=cols[10], time_string=cols[11], offense=cols[3], latitude=cols[7], longitude=cols[8])

CrimeDataT = noHeaderRdd.map(map_line)

CrimeDataT.take(10)

[CrimeData(date_string='6D', time_string='FIRST', offense='HOMICIDE', latitude='398943', longitude='133729'),
 CrimeData(date_string='4B', time_string='FOURTH', offense='SEX ABUSE', latitude='397769', longitude='144596'),
 CrimeData(date_string='1C', time_string='THIRD', offense='SEX ABUSE', latitude='396275', longitude='139402'),
 CrimeData(date_string='1C', time_string='THIRD', offense='SEX ABUSE', latitude='396518', longitude='139335'),
 CrimeData(date_string='2A', time_string='SECOND', offense='SEX ABUSE', latitude='395232', longitude='136881'),
 CrimeData(date_string='5C', time_string='FIFTH', offense='SEX ABUSE', latitude='402158.31', longitude='138824.53'),
 CrimeData(date_string='7B', time_string='SIXTH', offense='SEX ABUSE', latitude='402837', longitude='133810'),
 CrimeData(date_string='8D', time_string='SEVENTH', offense='SEX ABUSE', latitude='398794', longitude='127300'),
 CrimeData(date_string='5E', time_string='FIFTH', offense='HOMICIDE', latitude='400331', longitude='140

On obtient une liste de tuple plus lisible.

A présent, regroupez les données par type de crime (OFFENSE)

In [11]:
# Nous utiliserons la methode groupBY 

# soit on garde une approche par index et donc pas trés lisible
# grouped_by_offense_rdd = CrimeData.groupBy(lambda x: x[3])

# soit on utiliser les tuples
grouped_by_offense_rdd_tuple = CrimeDataT.groupBy(lambda x: x.offense)

In [12]:
#Affichez les 5 premiers éléments :
grouped_by_offense_rdd_tuple.take(5)

[('ASSAULT W/DANGEROUS WEAPON',
  <pyspark.resultiterable.ResultIterable at 0x7f5e14b72790>),
 ('ROBBERY', <pyspark.resultiterable.ResultIterable at 0x7f5e14bceee0>),
 ('THEFT F/AUTO', <pyspark.resultiterable.ResultIterable at 0x7f5e4047ee50>),
 ('BURGLARY', <pyspark.resultiterable.ResultIterable at 0x7f5e14bdb4f0>),
 ('HOMICIDE', <pyspark.resultiterable.ResultIterable at 0x7f5e14baaa30>)]

Ensuite, créez un RDD qui compte le nombre de chaque infraction (OFFENSE). 

In [13]:
#Indice : utilisez la méthode map sur le RDD pair obtenu précédement avec la function len

offense_counts = grouped_by_offense_rdd_tuple.map(lambda g: (g[0], len(g[1])))
offense_counts.take(5)

[('HOMICIDE', 104),
 ('BURGLARY', 3370),
 ('ROBBERY', 4071),
 ('THEFT F/AUTO', 10130),
 ('ASSAULT W/DANGEROUS WEAPON', 2309)]

A partir du RDD CrimeDataT, répéter l'opération de comptage des OFFENSES avec la méthode RDD groupByKey :

In [14]:
#Indice : utilisez la méthode map sur le RDD pair obtenu précédement avec la function len

offense_counts = CrimeDataT.map(lambda tuple: (tuple.offense, tuple) ).countByKey()

for offense, counts in offense_counts.items():
    print("{0:30s} {1:d}".format(offense, counts))
    

HOMICIDE                       104
SEX ABUSE                      299
THEFT/OTHER                    12904
BURGLARY                       3370
ROBBERY                        4071
THEFT F/AUTO                   10130
ASSAULT W/DANGEROUS WEAPON     2309
MOTOR VEHICLE THEFT            2675
ARSON                          35


Combien de meurtres (homicide) y a-t-il eu pendant la période couverte par les données? 

Attention votre traitement devra être insensible à la casse

In [15]:
# utilisez le RDD CrimeDataT pour compter le nombre d'homicide
# votre traitement devra être insensible à la maniére dont est ecrit homicide
# Vous devrez utiliser effectuer le comptage avec la méthode reduceByKey
# Vous afficherez le résultat de votre RDD homicide_count

homicide_count = CrimeDataT.filter(lambda crime: "homicide" in crime.offense.lower()).map(lambda crime: (crime.offense, 1)).reduceByKey(lambda a, b: a + b)

homicide_count.take(2)


[('HOMICIDE', 104)]

Pour un affichage plus sympa :

In [16]:
for method, count in homicide_count.collect():
    print("{0} {1:d}".format(method, count))

HOMICIDE 104


Affichez le top 5 des crimes :

In [17]:
weapon_counts = grouped_by_offense_rdd_tuple.map(lambda g: ( len(g[1]),g[0])).sortByKey(False)

weapon_counts.take(5)

[(12904, 'THEFT/OTHER'),
 (10130, 'THEFT F/AUTO'),
 (4071, 'ROBBERY'),
 (3370, 'BURGLARY'),
 (2675, 'MOTOR VEHICLE THEFT')]

Calculer la variance sur le nombre de crime :

Indice : vous pourrez utiliser la même approche faite dans le TP MapReduce

Vous pouvez à présent fermer le SparkContext.

In [18]:
sc.stop()